In [1]:
import os
import glob
import numpy as np
import xarray as xr
import rioxarray as riox
from datetime import datetime

In [2]:
yr = 2021

prefix = 'cper'
#prefix = 'redtop'

cluster_loc = 'hpc'

bm_units = 'lbs' #kgs for kg/ha or lbs for lb/acre
bm_unit_max = 5000

#epsg_out = "EPSG:32613" 
epsg_out = "EPSG:3857"

da_chunks = {'date': 50, 'y': 50, 'x': 50}

In [3]:
if cluster_loc == 'local':
    os.chdir(wkDIR)
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
    ncDIR = os.path.join('data/hls_nrt/', prefix)
    outDIR = 'data/gcloud/'
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    wkDIR = '/project/cper_neon_aop/hls_nrt/'
    ncDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix
    outDIR = '/90daydata/cper_neon_aop/hls_nrt/gcloud/'
    os.chdir(wkDIR)
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up cluster on HPC...')
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=16
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)

/project/cper_neon_aop/conda_envs/hls_nrt_env/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


   setting up cluster on HPC...


<Client: 'tcp://10.1.9.139:41062' processes=64 threads=128, memory=298.24 GiB>

In [4]:
if not os.path.exists(outDIR):
    os.mkdir(outDIR)

In [5]:
da_bare = riox.open_rasterio(os.path.join(ncDIR,
                                         'hls_cover',
                                          prefix + '_hls_BARE_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'}).chunk(da_chunks)
da_green = riox.open_rasterio(os.path.join(ncDIR,
                                         'hls_cover',
                                          prefix + '_hls_GREEN_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'}).chunk(da_chunks)
da_sd = riox.open_rasterio(os.path.join(ncDIR,
                                         'hls_cover',
                                          prefix + '_hls_SD_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'}).chunk(da_chunks)
da_litt = riox.open_rasterio(os.path.join(ncDIR,
                                         'hls_cover',
                                          prefix + '_hls_LITT_' + 
                                          str(yr) + '.nc')).rename({'time': 'date'}).chunk(da_chunks)
da_bm = riox.open_rasterio(os.path.join(ncDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_' +
                                        str(yr) + '.nc')).rename({'time': 'date'}).chunk(da_chunks)
da_bm_se = riox.open_rasterio(os.path.join(ncDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_se_' +
                                        str(yr) + '.nc')).rename({'time': 'date'}).chunk(da_chunks)
da_ndvi = riox.open_rasterio(os.path.join(ncDIR,
                                        'hls_ndvi',
                                        prefix + '_hls_ndvi_' +
                                        str(yr) + '.nc')).rename({'time': 'date'}).chunk(da_chunks)

if type(da_bare) ==  xr.core.dataset.Dataset:
    da_bare = da_bare['BARE']
    da_green = da_green['GREEN']
    da_sd = da_sd['SD']
    da_litt = da_litt['LITT']
    da_bm = da_bm['Biomass']
    da_bm_se = da_bm_se['Biomass_SE']
    da_ndvi = da_ndvi['NDVI']

elif type(da_bare) == xr.core.dataarray.DataArray:
    da_bare.name = 'BARE'
    da_green.name = 'GREEN'
    da_sd.name = 'SD'
    da_litt.name = 'LITT'
    da_bm.name = 'Biomass'
    da_bm_se.name = 'Biomass_SE'
    da_ndvi.name = 'NDVI'

if bm_units == 'lbs':
    # convert to lbs/acre
    da_bm = da_bm * 0.892179
    da_bm_se = np.log(np.exp(da_bm_se.chunk({'date': -1, 'y': 50, 'x': 50})  * 0.892179))

In [6]:
da_bare = da_bare.where(da_bare > 0.0, other=0.0)
da_bare = da_bare.where(da_bare < 1.0, other=1.0)

da_green = da_green.where(da_green > 0.0, other=0.0)
da_green = da_green.where(da_green < 1.0, other=1.0)

da_sd = da_sd.where(da_sd > 0.0, other=0.0)
da_sd = da_sd.where(da_sd < 1.0, other=1.0)

da_litt = da_litt.where(da_litt > 0.0, other=0.0)
da_litt = da_litt.where(da_litt < 1.0, other=1.0)

cov_mask = da_bare.notnull() & da_green.notnull() & da_sd.notnull() & da_litt.notnull()

In [7]:
ds = xr.merge([da_bare.where(cov_mask), 
               da_green.where(cov_mask), 
               da_sd.where(cov_mask),
               da_litt.where(cov_mask),
               da_bm.where((da_bm >= 0) & (da_bm < bm_unit_max)),
               da_bm_se.where((da_bm >= 0) & (da_bm < bm_unit_max)),
               da_ndvi.where((da_ndvi >= 0) & (da_ndvi <= 1))], 
             combine_attrs='drop')
ds = ds.where(ds != -9999.)

In [8]:
cov_sum = ds['BARE']+ds['SD']+ds['LITT']+ds['GREEN']
#cov_sum

In [9]:
ds['BARE'] = ds['BARE'] / cov_sum
ds['SD'] = ds['SD'] / cov_sum
ds['LITT'] = ds['LITT'] / cov_sum
ds['GREEN'] = ds['GREEN'] / cov_sum

In [10]:
#ds['date'] = [datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S') for x in ds['date'].values]

In [11]:
#ds['date'] = ds['date'].dt.date
ds['DRYVEG'] = ds['SD'] + ds['LITT']
ds['NONVEG'] = ds['BARE'] + ds['LITT']

In [12]:
da_bare.rio.crs

CRS.from_epsg(3857)

In [13]:
if ds.rio.crs is None:
    ds = ds.rio.write_crs(da_bare.rio.crs)

In [14]:
ds.attrs['_FillValue'] = 3.402823466385289e+38

In [15]:
ds.rio.reproject(epsg_out).to_netcdf(os.path.join(outDIR, 'hls_' + prefix + '_' + str(yr) + '_gcloud.nc'))